In [1]:
INPUT_PATH = r'..\data\processed\selected_features.csv'
MOVIES = r'..\models\prepared_features.pkl'
SIM_MATRIX = r'..\models\similarity_matrix.pkl'

# importing


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import ast
import nltk
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, cosine_distances
from sklearn.preprocessing import StandardScaler

import joblib
import requests

In [3]:
df = pd.read_csv(INPUT_PATH)
movies_df = joblib.load(MOVIES)
sim_martix = joblib.load(SIM_MATRIX)

In [4]:
movies_df = movies_df.drop('tags', axis=1)

In [5]:
movies_df

,movie_id,title,release_year,runtime,vote_average
0,19995,Avatar,2009,162.0,7.2
1,285,Pirates of the Caribbean: At World's End,2007,169.0,6.9
2,206647,Spectre,2015,148.0,6.3
3,49026,The Dark Knight Rises,2012,165.0,7.6
4,49529,John Carter,2012,132.0,6.1
...,...,...,...,...,...
4794,9367,El Mariachi,1992,81.0,6.6
4795,72766,Newlyweds,2011,85.0,5.9
4796,231617,"Signed, Sealed, Delivered",2013,120.0,7.0
4797,126186,Shanghai Calling,2012,98.0,5.7


In [6]:
sim_martix.head()

title,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter,Spider-Man 3,Tangled,Avengers: Age of Ultron,Harry Potter and the Half-Blood Prince,Batman v Superman: Dawn of Justice,...,On The Downlow,Sanctuary: Quite a Conundrum,Bang,Primer,Cavite,El Mariachi,Newlyweds,"Signed, Sealed, Delivered",Shanghai Calling,My Date with Drew
title,,,,,,,,,,,,,,,,,,,,,
Avatar,0.000000,6.855437,7.498988,6.924186,7.327051,7.745422,9.475129,7.036607,7.018364,7.796376,...,10.371754,14.025095,9.920004,10.429355,10.657899,10.741173,10.260152,8.582007,10.114592,10.296073
Pirates of the Caribbean: At World's End,6.855437,0.000000,7.827773,7.469110,8.305792,7.783211,9.951263,7.556566,7.408231,7.779427,...,10.591054,13.982609,10.224390,10.893415,10.980664,10.810721,10.831077,9.062726,10.329825,10.363444
Spectre,7.498988,7.827773,0.000000,7.939005,7.240326,7.340857,9.205136,6.988730,7.622658,6.809696,...,9.140086,13.060514,9.751758,10.308445,10.158242,10.220435,9.831296,8.214340,9.286020,9.694297
The Dark Knight Rises,6.924186,7.469110,7.939005,0.000000,8.704884,8.459863,9.502976,7.650207,7.320452,7.351813,...,10.453392,14.187843,10.126771,10.614140,10.654069,10.676247,10.824843,8.840840,10.016910,9.951507
John Carter,7.327051,8.305792,7.240326,8.704884,0.000000,6.860518,8.425543,6.756888,7.903302,7.217713,...,8.726927,12.591765,8.512029,9.446226,8.644103,8.728667,9.091770,7.788953,7.935028,8.647949


# get movie poster


In [19]:
def get_poster(movie_id):
    # movie_id = 285
    url = f"https://api.themoviedb.org/3/movie/{str(movie_id)}?api_key=c6f8304c6880c931e83d6234fe9d5095&include_adult=false&language=en-US&page=1"

    headers = {"accept": "application/json"}
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        poster_url = 'http://image.tmdb.org/t/p/w500' + \
            str(data['poster_path'])
    except:
        poster_url = 'not found'
    return poster_url

In [20]:
get_poster(19995)

'http://image.tmdb.org/t/p/w500/kyeqWdyUXW608qlYkRqosgbbJyK.jpg'

In [21]:
# movies_df['movie_poster'] = movies_df['movie_id'].apply(get_poster)

In [22]:
# joblib.dump(movies_df , r'..\models\movies_with_poster.pkl')

['..\\models\\movies_with_poster.pkl']

In [23]:
movies_df = joblib.load(r'..\models\movies_with_poster.pkl')

In [24]:
movies_df

,movie_id,title,release_year,runtime,vote_average,movie_poster
0,19995,Avatar,2009,162.0,7.2,http://image.tmdb.org/t/p/w500/kyeqWdyUXW608ql...
1,285,Pirates of the Caribbean: At World's End,2007,169.0,6.9,http://image.tmdb.org/t/p/w500/jGWpG4YhpQwVmjy...
2,206647,Spectre,2015,148.0,6.3,http://image.tmdb.org/t/p/w500/672kUEMtTHcaVYS...
3,49026,The Dark Knight Rises,2012,165.0,7.6,http://image.tmdb.org/t/p/w500/hr0L2aueqlP2BYU...
4,49529,John Carter,2012,132.0,6.1,http://image.tmdb.org/t/p/w500/lCxz1Yus07QCQQC...
...,...,...,...,...,...,...
4794,9367,El Mariachi,1992,81.0,6.6,http://image.tmdb.org/t/p/w500/zRh7K4SV1xQ419m...
4795,72766,Newlyweds,2011,85.0,5.9,http://image.tmdb.org/t/p/w500/zMEfv8533tJpNXw...
4796,231617,"Signed, Sealed, Delivered",2013,120.0,7.0,http://image.tmdb.org/t/p/w500/6BVCgmhLeSTF8nP...
4797,126186,Shanghai Calling,2012,98.0,5.7,http://image.tmdb.org/t/p/w500/2a1q1RTxspKxGWb...


In [25]:
movies_df['num_of_vote'] = df['vote_count']

In [26]:
movies_df = movies_df[['movie_id', 'title', 'release_year',
                       'runtime', 'vote_average', 'num_of_vote', 'movie_poster']]

In [27]:
movies_df

,movie_id,title,release_year,runtime,vote_average,num_of_vote,movie_poster
0,19995,Avatar,2009,162.0,7.2,11800,http://image.tmdb.org/t/p/w500/kyeqWdyUXW608ql...
1,285,Pirates of the Caribbean: At World's End,2007,169.0,6.9,4500,http://image.tmdb.org/t/p/w500/jGWpG4YhpQwVmjy...
2,206647,Spectre,2015,148.0,6.3,4466,http://image.tmdb.org/t/p/w500/672kUEMtTHcaVYS...
3,49026,The Dark Knight Rises,2012,165.0,7.6,9106,http://image.tmdb.org/t/p/w500/hr0L2aueqlP2BYU...
4,49529,John Carter,2012,132.0,6.1,2124,http://image.tmdb.org/t/p/w500/lCxz1Yus07QCQQC...
...,...,...,...,...,...,...,...
4794,9367,El Mariachi,1992,81.0,6.6,238,http://image.tmdb.org/t/p/w500/zRh7K4SV1xQ419m...
4795,72766,Newlyweds,2011,85.0,5.9,5,http://image.tmdb.org/t/p/w500/zMEfv8533tJpNXw...
4796,231617,"Signed, Sealed, Delivered",2013,120.0,7.0,6,http://image.tmdb.org/t/p/w500/6BVCgmhLeSTF8nP...
4797,126186,Shanghai Calling,2012,98.0,5.7,7,http://image.tmdb.org/t/p/w500/2a1q1RTxspKxGWb...


In [28]:
joblib.dump(movies_df, r'..\models\movies_with_poster.pkl')

['..\\models\\movies_with_poster.pkl']

---

# Sort popular movies


In [29]:
def get_top_n_movies(df, n):
    return df.sort_values(by=['num_of_vote', 'vote_average'], ascending=False).head(n)

In [30]:
get_top_n_movies(movies_df, 20)

,movie_id,title,release_year,runtime,vote_average,num_of_vote,movie_poster
96,27205,Inception,2010,148.0,8.1,13752,http://image.tmdb.org/t/p/w500/oYuLEt3zVCKq57q...
65,155,The Dark Knight,2008,152.0,8.2,12002,http://image.tmdb.org/t/p/w500/qJ2tW6WMUDux911...
0,19995,Avatar,2009,162.0,7.2,11800,http://image.tmdb.org/t/p/w500/kyeqWdyUXW608ql...
16,24428,The Avengers,2012,143.0,7.4,11776,http://image.tmdb.org/t/p/w500/RYMX2wcKCBAr24U...
788,293660,Deadpool,2016,108.0,7.4,10995,http://image.tmdb.org/t/p/w500/fSRb7vyIP8rQpL0...
95,157336,Interstellar,2014,169.0,8.1,10867,http://image.tmdb.org/t/p/w500/gEU2QniE6E77NI6...
287,68718,Django Unchained,2012,165.0,7.8,10099,http://image.tmdb.org/t/p/w500/7oWY8VDWW7thTzW...
94,118340,Guardians of the Galaxy,2014,121.0,7.9,9742,http://image.tmdb.org/t/p/w500/r7vmZjiyZw9rpJM...
426,70160,The Hunger Games,2012,142.0,6.9,9455,http://image.tmdb.org/t/p/w500/yXCbOiVDCxO71zI...
127,76341,Mad Max: Fury Road,2015,120.0,7.2,9427,http://image.tmdb.org/t/p/w500/8tZYtuWezp8Jbcs...


---


# Recommendation fun


In [31]:
movies_df['movie_id']

0        19995
1          285
2       206647
3        49026
4        49529
         ...  
4794      9367
4795     72766
4796    231617
4797    126186
4798     25975
Name: movie_id, Length: 4799, dtype: int64

In [32]:
sim_martix.columns = movies_df['movie_id']

In [33]:
sim_martix

movie_id,19995,285,206647,49026,49529,559,38757,99861,767,209112,...,182291,286939,124606,14337,67238,9367,72766,231617,126186,25975
title,,,,,,,,,,,,,,,,,,,,,
Avatar,0.000000,6.855437,7.498988,6.924186,7.327051,7.745422,9.475129,7.036607,7.018364,7.796376,...,10.371754,14.025095,9.920004,10.429355,10.657899,10.741173,10.260152,8.582007,10.114592,10.296073
Pirates of the Caribbean: At World's End,6.855437,0.000000,7.827773,7.469110,8.305792,7.783211,9.951263,7.556566,7.408231,7.779427,...,10.591054,13.982609,10.224390,10.893415,10.980664,10.810721,10.831077,9.062726,10.329825,10.363444
Spectre,7.498988,7.827773,0.000000,7.939005,7.240326,7.340857,9.205136,6.988730,7.622658,6.809696,...,9.140086,13.060514,9.751758,10.308445,10.158242,10.220435,9.831296,8.214340,9.286020,9.694297
The Dark Knight Rises,6.924186,7.469110,7.939005,0.000000,8.704884,8.459863,9.502976,7.650207,7.320452,7.351813,...,10.453392,14.187843,10.126771,10.614140,10.654069,10.676247,10.824843,8.840840,10.016910,9.951507
John Carter,7.327051,8.305792,7.240326,8.704884,0.000000,6.860518,8.425543,6.756888,7.903302,7.217713,...,8.726927,12.591765,8.512029,9.446226,8.644103,8.728667,9.091770,7.788953,7.935028,8.647949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
El Mariachi,10.741173,10.810721,10.220435,10.676247,8.728667,9.346231,8.602518,9.303843,9.627675,10.488703,...,8.161328,12.380700,6.965376,7.610457,6.474662,0.000000,8.649645,9.442151,7.806110,7.272100
Newlyweds,10.260152,10.831077,9.831296,10.824843,9.091770,9.415320,8.429076,9.766643,10.271965,9.958058,...,7.611727,11.477761,8.414413,8.073335,8.246677,8.649645,0.000000,7.973193,7.605728,7.629353
"Signed, Sealed, Delivered",8.582007,9.062726,8.214340,8.840840,7.788953,8.211931,7.978639,7.977890,8.180809,8.629944,...,8.261890,12.892368,8.813223,8.726143,8.836391,9.442151,7.973193,0.000000,8.175155,8.298198


In [68]:
# joblib.dump(sim_martix , SIM_MATRIX)

['..\\models\\similarity_matrix.pkl']

In [45]:
recomm_movies = pd.DataFrame(sim_martix[19995].sort_values(
    ascending=True).head(10)).reset_index()[1:]

In [46]:
recomm_movies

,title,19995
1,Independence Day,6.552219
2,Red Cliff,6.653749
3,The Curious Case of Benjamin Button,6.738605
4,Captain America: Civil War,6.798627
5,Watchmen,6.819373
6,Pirates of the Caribbean: At World's End,6.855437
7,The Hobbit: An Unexpected Journey,6.867764
8,American Gangster,6.870361
9,Transformers,6.918897


In [66]:
def recommmendation_by_id(movie_id):
    recomm_movies = pd.DataFrame(sim_martix[movie_id].sort_values(
        ascending=True).head(10)).reset_index()[1:]
    recom_df = pd.DataFrame(columns=movies_df.columns)
    for title in recomm_movies['title']:
        recom = movies_df[movies_df['title'] == title]
        recom_df = pd.concat([recom_df, recom])
    return recom_df

In [67]:
recommmendation_by_id(19995)

,movie_id,title,release_year,runtime,vote_average,num_of_vote,movie_poster
507,602,Independence Day,1996,145.0,6.7,3260,http://image.tmdb.org/t/p/w500/p0BPQGSPoSa8Ml0...
1298,12289,Red Cliff,2008,150.0,7.1,205,http://image.tmdb.org/t/p/w500/bxwuGaDXgEr2rm9...
100,4922,The Curious Case of Benjamin Button,2008,166.0,7.3,3292,http://image.tmdb.org/t/p/w500/26wEWZYt6yJkwRV...
26,271110,Captain America: Civil War,2016,147.0,7.1,7241,http://image.tmdb.org/t/p/w500/rAGiXaUfPzY7CDE...
163,13183,Watchmen,2009,163.0,7.0,2811,http://image.tmdb.org/t/p/w500/aZvOkdo203bm1kp...
1,285,Pirates of the Caribbean: At World's End,2007,169.0,6.9,4500,http://image.tmdb.org/t/p/w500/jGWpG4YhpQwVmjy...
98,49051,The Hobbit: An Unexpected Journey,2012,169.0,7.0,8297,http://image.tmdb.org/t/p/w500/yHA9Fc37VmpUA5U...
281,4982,American Gangster,2007,157.0,7.4,1502,http://image.tmdb.org/t/p/w500/sX9idXDqRUxE5ff...
111,1858,Transformers,2007,144.0,6.6,4040,http://image.tmdb.org/t/p/w500/lkZ9gqCEjzX85lK...
